In [3]:
import boto3
import json

In [6]:
# Initialize Step Functions client
sfn_client = boto3.client("stepfunctions")

# Define the state machine JSON
state_machine_definition = json.dumps(
    {
        "Comment": "State machine to trigger SageMaker Batch Transform",
        "StartAt": "StartBatchTransform",
        "States": {
            "StartBatchTransform": {
                "Type": "Task",
                "Resource": "arn:aws:states:::sagemaker:createTransformJob.sync",
                "Parameters": {
                    "TransformJobName": "BatchTransformJob-123",
                    "ModelName": "pipelines-ipiycr4j52a4-StockMarketPredictio-DX7H746r4M",
                    "TransformInput": {
                        "DataSource": {
                            "S3DataSource": {
                                "S3DataType": "S3Object",
                                "S3Uri": "s3://aws-portfolio-projects/snp500-data/inference_data/processed/sp500_processed.csv",
                            }
                        },
                        "ContentType": "text/csv",
                    },
                    "TransformOutput": {
                        "S3OutputPath": "s3://aws-portfolio-projects/snp500-data/batch-predictions/"
                    },
                    "TransformResources": {
                        "InstanceType": "ml.m5.large",
                        "InstanceCount": 1,
                    },
                },
                "End": True,
            }
        },
    }
)

In [5]:
# Create the state machine
response = sfn_client.create_state_machine(
    name="BatchTransformStateMachine",
    definition=state_machine_definition,
    roleArn="arn:aws:iam::930627915954:role/StepFunctionFullAccessRole",
)

print("Step Function Created:", response["stateMachineArn"])

In [10]:
# Update the state machine
response = sfn_client.update_state_machine(
    stateMachineArn="arn:aws:states:us-east-2:930627915954:stateMachine:BatchTransformStateMachine",  # Your state machine ARN
    definition=state_machine_definition,
)
print(f"State Machine Update Status: {response}")

State Machine Update Status: {'updateDate': datetime.datetime(2025, 2, 23, 22, 31, 12, 880000, tzinfo=tzlocal()), 'revisionId': '02621c73-0b2c-4d9d-897d-db57f9d3d14f', 'ResponseMetadata': {'RequestId': '9f59b1e3-37d4-4b3a-a84e-ff09ed85497b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9f59b1e3-37d4-4b3a-a84e-ff09ed85497b', 'date': 'Mon, 24 Feb 2025 04:33:01 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '82', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
